In [1]:
from math import *
import random

In [7]:
landmarks  = [[20.0, 20.0], [80.0, 80.0], [20.0, 80.0], [80.0, 20.0]]
world_size = 100.0

In [4]:
class robot:
    def __init__(self):
        self.x = random.random() * world_size
        self.y = random.random() * world_size
        self.orientation = random.random() * 2.0 * pi
        self.forward_noise = 0.0;
        self.turn_noise    = 0.0;
        self.sense_noise   = 0.0;
    
    def set(self, new_x, new_y, new_orientation):
        if new_x < 0 or new_x >= world_size:
#             raise ValueError, 'X coordinate out of bound'
            raise ValueError('X coordinate out of bound')
        if new_y < 0 or new_y >= world_size:
#             raise ValueError, 'Y coordinate out of bound'
            raise ValueError('Y coordinate out of bound')
        if new_orientation < 0 or new_orientation >= 2 * pi:
#             raise ValueError, 'Orientation must be in [0..2pi]'
            raise ValueError('Orientation must be in [0..2pi]')
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)
    
    def set_noise(self, new_f_noise, new_t_noise, new_s_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.forward_noise = float(new_f_noise);
        self.turn_noise    = float(new_t_noise);
        self.sense_noise   = float(new_s_noise);
    
    def sense(self):
        Z = []
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            dist += random.gauss(0.0, self.sense_noise)
            Z.append(dist)
        return Z
    
    def move(self, turn, forward):
        if forward < 0:
#             raise ValueError, 'Robot cant move backwards'
            raise ValueError('Robot cant move backwards')
        
        # turn, and add randomness to the turning command
        orientation = self.orientation + float(turn) + random.gauss(0.0, self.turn_noise)
        orientation %= 2 * pi
        
        # move, and add randomness to the motion command
        dist = float(forward) + random.gauss(0.0, self.forward_noise)
        x = self.x + (cos(orientation) * dist)
        y = self.y + (sin(orientation) * dist)
        x %= world_size    # cyclic truncate
        y %= world_size
        
        # set particle
        res = robot()
        res.set(x, y, orientation)
        res.set_noise(self.forward_noise, self.turn_noise, self.sense_noise)
        return res
    
    def Gaussian(self, mu, sigma, x):
        # calculates the probability of x for 1-dim Gaussian with mean mu and var. sigma
        return exp(- ((mu - x) ** 2) / (sigma ** 2) / 2.0) / sqrt(2.0 * pi * (sigma ** 2))
    
    def measurement_prob(self, measurement):
        # calculates how likely a measurement should be
        prob = 1.0;
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            prob *= self.Gaussian(dist, self.sense_noise, measurement[i])
        return prob

    def __repr__(self):
        return '[x=%.6s y=%.6s orient=%.6s]' % (str(self.x), str(self.y), str(self.orientation))

In [5]:
def eval(r, p):
    sum = 0.0;
    for i in range(len(p)): # calculate mean error
        dx = (p[i].x - r.x + (world_size/2.0)) % world_size - (world_size/2.0)
        dy = (p[i].y - r.y + (world_size/2.0)) % world_size - (world_size/2.0)
        err = sqrt(dx * dx + dy * dy)
        sum += err
    return sum / float(len(p))

In [19]:
# Make a robot called myrobot that starts at
# coordinates 30, 50 heading north (pi/2).
# Have your robot turn clockwise by pi/2, move
# 15 m, and sense. Then have it turn clockwise
# by pi/2 again, move 10 m, and sense again.
#
# Your program should print out the result of
# your two sense measurements.
#
# Don't modify the code below. Please enter
# your code at the bottom.
myrobot = robot()
myrobot.set(30, 50, pi/2)
print(myrobot)
myrobot = myrobot.move(-pi/2, 15)
print(myrobot)
print(myrobot.sense())
myrobot = myrobot.move(-pi/2, 10)
print(myrobot)
print(myrobot.sense())

[x=30.0 y=50.0 orient=1.5707]
[x=45.0 y=50.0 orient=0.0]
[39.05124837953327, 46.09772228646444, 39.05124837953327, 46.09772228646444]
[x=45.0 y=40.0 orient=4.7123]
[32.01562118716424, 53.150729063673246, 47.16990566028302, 40.311288741492746]


In [20]:
# Now add noise to your robot as follows:
# forward_noise = 5.0, turn_noise = 0.1,
# sense_noise = 5.0.
#
# Once again, your robot starts at 30, 50,
# heading north (pi/2), then turns clockwise
# by pi/2, moves 15 meters, senses,
# then turns clockwise by pi/2 again, moves
# 10 m, then senses again.
#
# Your program should print out the result of
# your two sense measurements.
#
forward_noise = 5.0
turn_noise = 0.1
sense_noise = 5.0

myrobot = robot()
myrobot.set_noise(forward_noise, turn_noise, sense_noise)
myrobot.set(30, 50, pi/2)
print(myrobot)
myrobot = myrobot.move(-pi/2, 15)
print(myrobot)
print(myrobot.sense())
myrobot = myrobot.move(-pi/2, 10)
print(myrobot)
print(myrobot.sense())

[x=30.0 y=50.0 orient=1.5707]
[x=46.418 y=49.543 orient=6.2553]
[40.093092838251714, 45.25367594349254, 41.533633345122915, 49.183145970201764]
[x=48.246 y=40.945 orient=4.9218]
[32.09750881477917, 42.649727704931244, 42.17232281766185, 41.51890843652686]


In [28]:
## initialize the particles
N = 1000
p = []
p = [robot() for i in range(N)]
print(len(p))
print(p[:10])

1000
[[x=5.3373 y=79.179 orient=5.5605], [x=14.686 y=30.003 orient=0.4388], [x=15.629 y=4.0468 orient=0.4559], [x=54.225 y=72.846 orient=4.4675], [x=46.926 y=91.473 orient=4.3314], [x=50.731 y=42.194 orient=1.2906], [x=94.134 y=97.314 orient=4.4541], [x=5.6488 y=20.620 orient=5.6505], [x=73.911 y=83.945 orient=3.8570], [x=66.042 y=80.825 orient=6.2072]]


In [29]:
# Now we want to simulate robot
# motion with our particles.
# Each particle should turn by 0.1
# and then move by 5. 
p = [pi.move(0.1, 5) for pi in p]
print(p[:10])

[[x=9.3991 y=76.263 orient=5.6605], [x=18.978 y=32.569 orient=0.5388], [x=19.876 y=6.6856 orient=0.5559], [x=53.503 y=67.898 orient=4.5675], [x=45.539 y=86.669 orient=4.4314], [x=51.627 y=47.113 orient=1.3906], [x=93.347 y=92.377 orient=4.5541], [x=9.9561 y=18.081 orient=5.7505], [x=70.483 y=80.305 orient=3.9570], [x=71.040 y=80.945 orient=0.0240]]


In [33]:
# Now we want to give weight to our 
# particles. This program will print a
# list of 1000 particle weights.
myrobot = robot()
myrobot.move(0.1, 5)
Z = myrobot.sense()   # this is the measurement, more precisely, the robot's measurements

N = 1000
particles = []
particles = [robot() for i in range(N)]
for p in particles:
    p.set_noise(0.05, 0.05, 5.0)

p2 = []
for i in range(N):
    p2.append(particles[i].move(0.1, 5.0))
particles = p2

p_weights = []
p_weights = [p.measurement_prob(Z) for p in particles]
print(p_weights[:10])

[7.368528909931213e-119, 2.1426600854570977e-56, 1.0472383107370832e-43, 4.8180086498278436e-71, 1.3556935703556169e-49, 1.5951233647517637e-17, 4.898636734090351e-78, 1.3550855946787636e-116, 6.412450240518966e-101, 4.8473499629716523e-85]
